In [1]:
from datasets import load_dataset, load_metric, load_from_disk, Audio
from datasets import ClassLabel, DatasetDict, Dataset
import random
import pandas as pd
from IPython.display import display, HTML
import re
import json
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
import IPython.display as ipd
import numpy as np
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import Wav2Vec2ForCTC
from transformers import TrainingArguments
from transformers import Trainer
from evaluate import load
import os
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
import argparse
import json
import logging
import os
import subprocess
import tarfile
import urllib.request

Dataset

In [2]:
#gale_speech = load_dataset('csv', data_files=['train13.csv', 'train14.csv'])
gale_speech = load_dataset('csv', data_files=['short_train_2013.csv', 'short_train_2014.csv'])
gale_speech = gale_speech.rename_column('transcript', 'text')
gale_speech

Using custom data configuration default-a53efedd5fb11b01
Found cached dataset csv (/home/ujan/.cache/huggingface/datasets/csv/default-a53efedd5fb11b01/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['audio_path', 'text', 'duration'],
        num_rows: 107016
    })
})

In [3]:
dataset = gale_speech.cast_column("audio_path", Audio())
dataset = dataset.rename_column('audio_path', 'audio')
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))  # downsample to 16k
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'duration'],
        num_rows: 107016
    })
})

In [4]:
#dataset = dataset.filter(lambda example: example['audio'] is not None)
dataset = dataset.filter(lambda example: example['text'] is not None)
dataset

Loading cached processed dataset at /home/ujan/.cache/huggingface/datasets/csv/default-a53efedd5fb11b01/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-e66b721c73199440.arrow


DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'duration'],
        num_rows: 105182
    })
})

Split into train test

In [5]:
dataset = dataset['train'].train_test_split(test_size=0.1)  # 178, 20
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'duration'],
        num_rows: 94663
    })
    test: Dataset({
        features: ['audio', 'text', 'duration'],
        num_rows: 10519
    })
})

In [6]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(dataset["train"].remove_columns(["duration", "audio"]))

,text
0,这首歌讲述了中华儿女对党和祖国的热爱之情就像对身边的亲人的那种感情一样是发自内心的
1,嗯
2,你选你选举是其中一个一个方面
3,一是简单不铺张
4,禁止向朝鲜运送可能被用做军事目的的设备
5,贪渎的陈水扁终结民进党的台湾共和国这个政权
6,看到丈夫写下了这样的字据张红感到十分生气因为丈夫根本就没和她商量过这件事
7,感谢收看今天的军事报道
8,凯西强调
9,这个不可能吧


Get vocab

In [7]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'  ######## chinese?

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

dataset = dataset.map(remove_special_characters)

  0%|          | 0/94663 [00:00<?, ?ex/s]

  0%|          | 0/10519 [00:00<?, ?ex/s]

In [8]:
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocabs = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

We use the json file to instantiate an object of the Wav2Vec2CTCTokenizer class.

In [9]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'劫': 0,
 '嘣': 1,
 '梓': 2,
 '鼎': 3,
 '宰': 4,
 '斋': 5,
 '持': 6,
 '己': 7,
 '缚': 8,
 '台': 9,
 '橄': 10,
 '绳': 11,
 '卵': 12,
 '然': 13,
 '韩': 14,
 '赓': 15,
 '漳': 16,
 '逐': 17,
 '昊': 18,
 '畜': 19,
 '雷': 20,
 '俞': 21,
 '冦': 22,
 '卯': 23,
 '询': 24,
 '曝': 25,
 '尖': 26,
 '料': 27,
 '属': 28,
 '炜': 29,
 '赢': 30,
 '梆': 31,
 '抨': 32,
 '咪': 33,
 '釜': 34,
 '灿': 35,
 '画': 36,
 '麻': 37,
 '羊': 38,
 '虎': 39,
 '寂': 40,
 '窃': 41,
 '堵': 42,
 '除': 43,
 '讽': 44,
 '戴': 45,
 '深': 46,
 '玛': 47,
 '玻': 48,
 '可': 49,
 '奋': 50,
 '比': 51,
 '崇': 52,
 '讯': 53,
 '圃': 54,
 '倔': 55,
 '榨': 56,
 '誓': 57,
 '疵': 58,
 '须': 59,
 '斐': 60,
 '像': 61,
 '霉': 62,
 '窒': 63,
 '罗': 64,
 '门': 65,
 '通': 66,
 '路': 67,
 '蝎': 68,
 '话': 69,
 '捡': 70,
 '拗': 71,
 '逻': 72,
 '户': 73,
 '吟': 74,
 '昨': 75,
 '藉': 76,
 '彬': 77,
 '拿': 78,
 '伊': 79,
 '飘': 80,
 '增': 81,
 '矿': 82,
 '仰': 83,
 '枝': 84,
 '臀': 85,
 '境': 86,
 '斑': 87,
 '哀': 88,
 '紫': 89,
 '桦': 90,
 '盛': 91,
 '苇': 92,
 '扑': 93,
 '粤': 94,
 '磕': 95,
 '触': 96,
 '遛': 97,
 '橛': 98,
 '眶': 99,
 '榭': 100,

Wav2Vec2 feature extractor

In [10]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

4110


To make the usage of Wav2Vec2 as user-friendly as possible, the feature extractor and tokenizer are wrapped into a single Wav2Vec2Processor class so that one only needs a model and processor object

In [31]:
with open('gale_speech_vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [2]:
tokenizer = Wav2Vec2CTCTokenizer("./gale_speech_vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [3]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [4]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [14]:
dataset['train'][0]

{'audio': {'path': 'gale_p3_cmn_bc_speech_p1/slices/CCTVNEWS_PEOPLEINNEWS_CMN_20070416_215701_392.wav',
  'array': array([ 2.5102904e-04,  4.1338913e-06, -8.0837542e-04, ...,
         -3.0592349e-03, -1.4182443e-03,  0.0000000e+00], dtype=float32),
  'sampling_rate': 16000},
 'text': '现在呢就说是在完成她的爱好',
 'duration': 4.015}

In [15]:
rand_int = random.randint(0, len(dataset['train'])-1)

print(dataset['train'][rand_int]["text"])
ipd.Audio(data=dataset['train'][rand_int]["audio"]["array"], autoplay=True, rate=16000)

我说你怎么写了这个干什么呀我说人家留学人多呢你写这个干什么呀


In [37]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=4)

#2:   0%|          | 0/23666 [00:00<?, ?ex/s]

#1:   0%|          | 0/23666 [00:00<?, ?ex/s]

#0:   0%|          | 0/23666 [00:00<?, ?ex/s]

#3:   0%|          | 0/23665 [00:00<?, ?ex/s]

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your l

In [ ]:
dataset

In [2]:
dataset.save_to_disk('/home/ujan/Downloads/Gale/gale_speech_vectorized')

NameError: name 'dataset' is not defined

In [5]:
dataset = load_from_disk('/home/ujan/Downloads/Gale/gale_speech_vectorized')

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


In [6]:
demo = DatasetDict()
demo['train'] = dataset['train'].select([i for i in range(10000)])
demo['val'] = dataset['test'].select([i for i in range(1000)])

Training & Evaluation

In [7]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [8]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [9]:
#wer = load("wer")
cer_metric = load("/home/ujan/Downloads/evaluate/metrics/cer/cer.py")

In [10]:
model = Wav2Vec2ForCTC.from_pretrained(
    "wav2vec2-xls-r-300m", 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

In [11]:
#model.save_pretrained('wav2vec2-xls-r-300m')

The first component of Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretrainind and as stated in the paper does not need to be fine-tuned anymore 

In [12]:
model.freeze_feature_encoder()

In [13]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    #wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [14]:
training_args = TrainingArguments(
    output_dir='/home/ujan/Notebooks/aishell_ctc',
    group_by_length=True,
    #per_device_train_batch_size=32,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_accumulation_steps=4,
    num_train_epochs=10,
    fp16=True,
    gradient_checkpointing=True, 
    save_steps=500, #
    eval_steps=500, #
    logging_steps=500, #
    learning_rate=0.0002,
    weight_decay=0.005,
    warmup_steps=500,
    save_total_limit=2,
    dataloader_num_workers=0,
)

In [15]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=demo["train"],
    eval_dataset=demo["val"], # validation
    tokenizer=processor.feature_extractor,
)

Using cuda_amp half precision backend


In [16]:
trainer.train()

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10000
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 6250
  Number of trainable parameters = 315441294
/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warni

Step,Training Loss,Validation Loss,Cer
500,30.258900,8.827227,1.000000
1000,6.662100,61.278053,1.002093


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to /home/ujan/Notebooks/aishell_ctc/checkpoint-500
Configuration saved in /home/ujan/Notebooks/aishell_ctc/checkpoint-500/config.json
Model weights saved in /home/ujan/Notebooks/aishell_ctc/checkpoint-500/pytorch_model.bin
Feature extractor saved in /home/ujan/Notebooks/aishell_ctc/checkpoint-500/preprocessor_config.json
Deleting older checkpoint [/home/ujan/Notebooks/aishell_ctc/checkpoint-2000] due to args.save_total_limit
/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoin

KeyboardInterrupt: 

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 109973
    })
    test: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 12220
    })
})

In [22]:
len(dataset['train'][23]['input_values'])

92529

In [ ]:
aishell = load_

Evaluation

In [15]:
device = torch.device("cuda")
model.to(device)

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (2): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (3): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (4): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)

In [16]:
def map_to_result(batch):
    with torch.no_grad():
        input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]
    batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
    return batch

results = dataset["test"].map(map_to_result, remove_columns=dataset["test"].column_names)

  0%|          | 0/7176 [00:00<?, ?ex/s]

In [17]:
print("Test CER: {:.3f}".format(cer.compute(predictions=results["pred_str"], references=results["text"])))

Test CER: 7.120


In [18]:
results_val = dataset["validation"].map(map_to_result, remove_columns=dataset["test"].column_names)

  0%|          | 0/14326 [00:00<?, ?ex/s]

In [24]:
print("Val CER: {:.3f}".format(cer.compute(predictions=results_val["pred_str"], references=results_val["text"])))

Val CER: 6.723
